In [39]:
###build a basic chatbot using langgraph

In [40]:
%pip install langgraph
import os
os.environ["GROQ_API_KEY"] = "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"
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END, add_messages


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [41]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages:Annotated[list,add_messages]

In [42]:
# Create a .env file with your GROQ API key

api_key = "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"  # Replace with your key

with open(".env", "w") as f:
    f.write(f"GROQ_API_KEY={api_key}\n")

print("✅ .env file created successfully!")


✅ .env file created successfully!


In [43]:

import os
from dotenv import load_dotenv
load_dotenv()
print("Environment variables loaded from .env file.",os.getenv("GROQ_API_KEY"))
print("GROQ_API_KEY:", os.getenv("TAVILY_API_KEY"))

Environment variables loaded from .env file. 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
GROQ_API_KEY: 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


In [44]:
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

llm=ChatGroq(model="llama3-8b-8192")

In [45]:
###chatbot tool

In [46]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021BBF693080>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021BBF691730>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [47]:
llm=init_chat_model("groq:llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021BBF613FE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021BBF613D40>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [48]:
## Node function to handle the chatbot logic
os.environ["GROQ_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] ="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"
print("GROQ_API_KEY:", os.getenv("TAVILY_API_KEY"))
def chatbot(state: State):
    return {"meesages":[llm.invoke(state["messages"])]}

    

GROQ_API_KEY: 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


In [49]:
graph_builder = StateGraph(State) 
## Adding Node to the graph
graph_builder.add_node("llmchatbot",chatbot)
## adding edges
graph_builder.add_edge(START, "llmchatbot")
graph_builder.add_edge("llmchatbot", END)
##compile the grpah 
graph = graph_builder.compile()

In [50]:
# visulaise the graph
%pip install IPython

from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Error displaying graph: {e}")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Error displaying graph: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [51]:
response=graph.invoke({"messages":"hi how are you tell me ur name?"})

In [52]:

response["messages"][-1].content # Access the first message in the response
# Display the response

'hi how are you tell me ur name?'

In [53]:
for event in graph.stream({"messages":"Hi How are you?"}):
    for value in event.values():
        if value is not None and "messages" in value and value["messages"]:
            print(value["messages"][-1].content)

In [54]:
###chatbot tool

In [55]:
%pip install langchain_tavily

from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=2)
tool.invoke("What is langgraph")


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


{'error': requests.exceptions.ConnectionError(urllib3.exceptions.MaxRetryError('HTTPSConnectionPool(host=\'api.tavily.com\', port=443): Max retries exceeded with url: /search (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000021BBF5D1970>: Failed to resolve \'api.tavily.com\' ([Errno 11001] getaddrinfo failed)"))'))}

In [57]:
import os
print("Key:", os.getenv("TAVILY_API_KEY"))


Key: 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
